In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaggle-survey-2021/kaggle_survey_2021_responses.csv
/kaggle/input/kaggle-survey-2021/supplementary_data/kaggle_survey_2021_methodology.pdf
/kaggle/input/kaggle-survey-2021/supplementary_data/kaggle_survey_2021_answer_choices.pdf


In [2]:
# load dataset
data = pd.read_csv("../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv", low_memory = False)

In [3]:
# first five rows
data.head()

,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,...,Q38_B_Part_3,Q38_B_Part_4,Q38_B_Part_5,Q38_B_Part_6,Q38_B_Part_7,Q38_B_Part_8,Q38_B_Part_9,Q38_B_Part_10,Q38_B_Part_11,Q38_B_OTHER
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,For how many years have you been writing code ...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,...,"In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor..."
1,910,50-54,Man,India,Bachelor’s degree,Other,5-10 years,Python,R,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,784,50-54,Man,Indonesia,Master’s degree,Program/Project Manager,20+ years,NaN,NaN,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,924,22-24,Man,Pakistan,Master’s degree,Software Engineer,1-3 years,Python,NaN,NaN,...,NaN,NaN,TensorBoard,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,575,45-49,Man,Mexico,Doctoral degree,Research Scientist,20+ years,Python,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [4]:
# size of the dataset
data.shape # 25973 rows, 369 columns

(25974, 369)

In [5]:
def rename_columns(df):
    """
    input: the dataset we want to rename the columns
    output: combine the first row of the dataset into the original column
    """
    original_columns = df.columns
    num_col = df.shape[1]
    first_row = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data below the first row
    # create a list containing new column names
    new_cols = []
    for col in range(num_col):
        new_col_name = original_columns[col] + '_' + first_row[col]
        new_cols.append(new_col_name)
    df.columns = new_cols # assign the new column names to the dataset
    return df

In [6]:
def replace_nan(df):
    """
    input:
    df - the target dataset
    
    output:
    a new dataset with nan values replaced as 0 and non-nan values replaced with 1
    """
    array = np.where(df.isnull(),0,1)
    df = pd.DataFrame(data=array, columns=df.columns)
    
    return df


#role_data = replace_nan(role_question)

In [7]:
renamed_data = rename_columns(data)

In [8]:
renamed_data.columns[:20]

Index(['Time from Start to Finish (seconds)_Duration (in seconds)',
       'Q1_What is your age (# years)?',
       'Q2_What is your gender? - Selected Choice',
       'Q3_In which country do you currently reside?',
       'Q4_What is the highest level of formal education that you have attained or plan to attain within the next 2 years?',
       'Q5_Select the title most similar to your current role (or most recent title if retired): - Selected Choice',
       'Q6_For how many years have you been writing code and/or programming?',
       'Q7_Part_1_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python',
       'Q7_Part_2_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R',
       'Q7_Part_3_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL',
       'Q7_Part_4_What programming languages do you use on a regular basis? (Select 

In [9]:
renamed_data.head()

,Time from Start to Finish (seconds)_Duration (in seconds),Q1_What is your age (# years)?,Q2_What is your gender? - Selected Choice,Q3_In which country do you currently reside?,Q4_What is the highest level of formal education that you have attained or plan to attain within the next 2 years?,Q5_Select the title most similar to your current role (or most recent title if retired): - Selected Choice,Q6_For how many years have you been writing code and/or programming?,Q7_Part_1_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python,Q7_Part_2_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R,Q7_Part_3_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL,...,"Q38_B_Part_3_In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments? (Select all that apply) - Selected Choice - Comet.ml","Q38_B_Part_4_In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments? (Select all that apply) - Selected Choice - Sacred + Omniboard","Q38_B_Part_5_In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments? (Select all that apply) - Selected Choice - TensorBoard","Q38_B_Part_6_In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments? (Select all that apply) - Selected Choice - Guild.ai","Q38_B_Part_7_In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments? (Select all that apply) - Selected Choice - Polyaxon","Q38_B_Part_8_In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments? (Select all that apply) - Selected Choice - ClearML","Q38_B_Part_9_In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments? (Select all that apply) - Selected Choice - Domino Model Monitor","Q38_B_Part_10_In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments? (Select all that apply) - Selected Choice - MLflow","Q38_B_Part_11_In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments? (Select all that apply) - Selected Choice - None","Q38_B_OTHER_In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments? (Select all that apply) - Selected Choice - Other"
1,910,50-54,Man,India,Bachelor’s degree,Other,5-10 years,Python,R,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,784,50-54,Man,Indonesia,Master’s degree,Program/Project Manager,20+ years,NaN,NaN,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,924,22-24,Man,Pakistan,Master’s degree,Software Engineer,1-3 years,Python,NaN,NaN,...,NaN,NaN,TensorBoard,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,575,45-49,Man,Mexico,Doctoral degree,Research Scientist,20+ years,Python,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
5,781,45-49,Man,India,Doctoral degree,Other,< 1 years,Python,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
demo_data = renamed_data.iloc[: , 1:7]

In [11]:
demo_data['participant_id'] = range(0, demo_data.shape[0])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
def pivot_col(df, col , aggr = 'participant_id'):
    """
    input:
    df - target dataset
    col - the column we want to pivot its value as new columns
    aggr - the column we used to group by the dataset
    
    output:
    return a pivoted dataframe where columns are value from the col of old dataframe
    """
    
    pivoted_df = df.pivot(index = aggr, columns=col, values=col).reset_index().iloc[: , 1:]
    pivoted_df.index = pivoted_df.index + 1
    return pivoted_df

In [14]:
def pivot_df(df, var_cols, aggr = 'participant_id'):
    """
    input:
    df - targer dataframe
    var_cols - a list of column names we want to pivot
    aggr - the column we used to group by the dataset
        
    output:
    return a dataframe where each column comes from value of each col of old dataframe
    NaN value replaced with 0 while non-NaN value replaced with 1
    """
    
    pivoted_df = []
    for col in var_cols:
        pivoted = pivot_col(df, col, aggr)
        pivoted_df.append(pivoted)
        #pivoted_df = pd.merge(pivoted, pivoted, left_index=True, right_index=True)  
    pivoted_df = pd.concat(pivoted_df, axis=1, ignore_index=False)
    pivoted_df = replace_nan(pivoted_df)
    #pivoted_df['participant_id'] = pivoted_df.index
    pivoted_df.index = pivoted_df.index + 1
    return pivoted_df
demo_data_matrix = pivot_df(demo_data, demo_data.columns[:-1])

In [15]:
demo_data_matrix

,18-21,22-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-69,...,Software Engineer,Statistician,Student,1-3 years,10-20 years,20+ years,3-5 years,5-10 years,< 1 years,I have never written code
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25969,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
25970,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
25971,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
25972,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
part_data = renamed_data.iloc[: , 7:]
part_data = replace_nan(part_data)
part_data.index = part_data.index + 1
dfs = [demo_data_matrix, part_data]
data_pivoted = pd.concat(dfs, axis=1, ignore_index=False)

In [17]:
data_matrix = data_pivoted.dot(np.transpose(data_pivoted))

In [18]:
data_matrix

,1,2,3,4,5,6,7,8,9,10,...,25964,25965,25966,25967,25968,25969,25970,25971,25972,25973
1,62,20,21,26,27,20,5,33,15,21,...,26,21,22,26,32,35,7,11,17,10
2,20,48,22,21,21,15,9,19,17,22,...,20,17,15,17,19,21,8,16,18,10
3,21,22,54,19,17,17,9,20,13,20,...,21,22,12,16,20,22,10,20,13,8
4,26,21,19,49,20,19,5,25,15,20,...,19,18,20,24,16,27,9,17,15,13
5,27,21,17,20,50,17,7,21,13,23,...,25,14,17,23,20,27,8,9,18,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25969,35,21,22,27,27,18,8,32,16,25,...,27,20,26,30,39,77,9,10,17,10
25970,7,8,10,9,8,7,7,7,10,10,...,7,10,10,9,6,9,20,1,10,3
25971,11,16,20,17,9,13,0,10,5,10,...,9,15,1,7,8,10,1,54,5,12
25972,17,18,13,15,18,11,6,17,11,17,...,15,14,15,17,15,17,10,5,26,5


In [21]:
def closest_participant(participant_id, participant_matrix):
    """
    input:
    participant_id - target participant
    participant_matrix - matrix where shows the similarity between each participant
    
    output - the list of participants other than the target participant, ranked by similarity
    """
    participant_list = participant_matrix[[participant_id]]
    participant_list = participant_list.sort_values(by = participant_id, ascending = False)
    
    return participant_list.index[1:]



In [86]:
closest_participant(5,data_matrix)

Int64Index([22860, 20119, 20409,  1282, 14366, 20739,  3998, 19188,  1591,
            22013,
            ...
            15592,  6362,  9632,  4854, 19986, 25507,  6672, 18980,  3292,
              361],
           dtype='int64', length=25972)

In [87]:
def compensation(df , participant):
    """
    input -
    df - target dataset
    participant - the id of the participant
    
    output -
    the yearly compensation of that participant
    """
    
    compensation = df.loc[df.index == participant]['Q25_What is your current yearly compensation (approximate $USD)?'].iloc[0]
    
    return compensation

compensation(renamed_data,5)

'30,000-39,999'

In [88]:
def similar_user_compensation(df, participant_ids):
    """
    input:
    df - target dataset
    participant_ids - a list of participant ids
    output:
    the first participant id that has non-null compensation data
    """
    for participant in participant_ids:
        if compensation(df, participant) is not None:
            return compensation(df, participant)
        else:
            pass
similar_user_compensation(renamed_data,closest_participant(5,data_matrix))

'$0-999'

In [28]:
test = renamed_data.loc[renamed_data.index.isin([2000,20567])]

In [92]:
test.iloc[: , 1:10]

,Q1_What is your age (# years)?,Q2_What is your gender? - Selected Choice,Q3_In which country do you currently reside?,Q4_What is the highest level of formal education that you have attained or plan to attain within the next 2 years?,Q5_Select the title most similar to your current role (or most recent title if retired): - Selected Choice,Q6_For how many years have you been writing code and/or programming?,Q7_Part_1_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python,Q7_Part_2_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R,Q7_Part_3_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL
2000,30-34,Woman,Japan,Bachelor’s degree,Machine Learning Engineer,3-5 years,Python,NaN,NaN
20567,45-49,Man,Poland,Doctoral degree,Data Scientist,3-5 years,Python,R,NaN


In [25]:
compensation(renamed_data,20567)

'5,000-7,499'

In [ ]:
renamed_data.loc[renamed_data.index == 2000].iloc[:,50:60]

In [ ]:
renamed_data.loc[renamed_data.index == 20567].iloc[:,50:60]

In [ ]:
"""
def compute_correlation(df, user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    df - dataset where is a matrix of user and their pivoted answer columns
    OUTPUT
    the correlation between the matching ratings between the two users
    '''
    answer_1 = list(df.loc[df.index == user1].iloc[0])
    answer_2 = list(df.loc[df.index == user2].iloc[0])
    
    dot_product = np.vdot(answer_1, answer_2)
    
    return dot_product #return the correlation
"""

In [ ]:
def subset_data(df, col, criteria):
    """
    input:
    df: the dataset we want to subset from
    col: target columns as the filter
    criteria: value to feed the filter
    
    output:
    a new dataset which is a subset of the original one
    """
    
    new_df = df.loc[df[col] == criteria]
    
    return new_df
"""
ds_data = subset_data(renamed_data, 
           'Q5_Select the title most similar to your current role (or most recent title if retired): - Selected Choice',
           'Data Scientist')
"""

In [ ]:
def question_columns(df, query, method = 'strict'):
    """
    input: 
    df - target dataset
    query - str, query we want to find relevant infomation in the dataset. e.g. 'Q7', or 'machine learning' 
    
    output:
    a subset of data which include the columns of the query in interest
    
    method:
    if it == strict, which means we will look for the question exactly EQUALS to the query. e.g. if we search 'age', then 'language' won't
    be taken into account in this case;
    
    if it == loose, which means we will look for the question exactly CONTAINS the query. e.g. if we search 'age', then 'language' will
    be taken into account in this case.
    """
    columns = df.columns
    question_col = []
    for col in columns:
        if method == 'strict':
            col_parts = col.lower().split() # each column name will be separated into single word tokens at first
            if query.lower() in col_parts:
                question_col.append(col)
        elif method == 'loose':
            if query.lower() in col.lower():
                question_col.append(col)
    return df[question_col]

In [ ]:
question_columns(renamed_data, query = 'compensation', method = 'strict')

## What questions we want to ask here?
1. How many types of data scientists based on activity they usually do?
2. Which skill each type emphasizes at?
3. Which platform/tools each type emphazies at?
4. Demographic segmentation of each type?
5. Profession segmentation of each type?

In [ ]:
#feature_data = role_data.iloc[: , 1:]

In [ ]:
"""
def kmeans_cluster_opt(df, init = 'k-means++', max_num_cluster = 9):
    
    input: 
    df - the dataset we want to segments into cluster
    init - the way we want to initialize the starting centroid
    max_num_cluster - the max number of cluster
    
    output:
    a visualization showing the line graph indicating the optimal number of klusters, based on inertias value
    
    num_clusters = list(range(1, max_num_cluster))
    inertias = []

    for k in num_clusters:
        model = KMeans(init=init, n_clusters=k, random_state = 42)
        model.fit(df)
        inertias.append(model.inertia_)

    
    plt.plot(num_clusters, inertias, '-o')

    plt.xlabel('number of clusters (k)')
    plt.ylabel('inertia')

    plt.show()
"""

In [ ]:
#kmeans_cluster_opt(feature_data)

In [ ]:
"""
def kmeans_predict(df, init = 'k-means++', n_clusters = 4):
    
    input:
    df - dataset we want to segment into clusters
    init - the way we want to initialize the starting centroid
    n_clusters - the number of cluster
    
    output:
    labels - return an array of predictions on the cluster label of given features
    centers - centroid values of each cluster
    model = KMeans(init=init, n_clusters = n_clusters, random_state = 42)

    model.fit(df)

    labels = model.predict(df)
    
    centers = np.array(model.cluster_centers_)
    
    return labels, centers
"""